# _SQL_ y libros
_Structured query language_ o _SQL_ es —como nos dice su nombre— un lenguaje de programación estructurado que nos permite hacer consultas en bases de datos. Es fundamental en el mundo de los datos y por eso dedicaremos el proyecto de hoy a él. Haremos algunas consultas elementales y explicaremos algunos conceptos clave. Vamos a trabajar con tres tablas relacionales: autores, préstamos y libros. Éstas forman parte de la pequeña base de datos de una biblioteca ficticia que tiene libros de mis autores preferidos. A divertirnos.

## Instalación de `sqlalchemy` y creación de un _engine_
Primero debemos instalar `sqlalchemy` con `pip` para poder hacer consultas _SQL_ en _Python_

In [1]:
!pip install sqlalchemy pandas
import pandas as pd
from sqlalchemy import create_engine, text

Con `sqlalchemy` crearemos un _engine_, que es un puente para establecer la conexión entre el código y la base de datos.

In [2]:
# Nuestro `engine` o puente.
engine = create_engine('sqlite:///biblioteca.db')

Ahora que tenemos nuestro puente vamos a crear las tablas. Determinaremos los campos (columnas) y el tipo de variable de cada uno de ellos.

In [3]:
with engine.begin() as conn:
    # Esto es un pequeño reinicio necesario para que el proyecto funcione.
    conn.execute(text("DROP TABLE IF EXISTS Prestamos"))
    conn.execute(text("DROP TABLE IF EXISTS Libros"))
    conn.execute(text("DROP TABLE IF EXISTS Autores"))

    # Creación de las tablas.
    conn.execute(text("""
        CREATE TABLE Autores (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            pais TEXT
        )
    """))
    conn.execute(text("""
        CREATE TABLE Libros (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titulo TEXT NOT NULL,
            autor_id INTEGER,
            año_publicacion INTEGER,
            genero TEXT,
            FOREIGN KEY(autor_id) REFERENCES Autores(id)
        )
    """))
    conn.execute(text("""
        CREATE TABLE Prestamos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            libro_id INTEGER,
            fecha_prestamo DATE,
            fecha_devolucion DATE,
            usuario TEXT,
            FOREIGN KEY(libro_id) REFERENCES Libros(id)
        )
    """))


Hemos creado las tablas con sus respectivos campos, pero ahora debemos darles registros (datos).

In [4]:
with engine.begin() as conn:
    # Inserta autores.
    conn.execute(text("""
        INSERT INTO Autores (nombre, pais) VALUES 
        ('Tryno Maldonado', 'México'),
        ('Daniel Espartaco Sánchez', 'México'),
        ('José Emilio Pacheco', 'México'),
        ('Rafael Bernal', 'México')
    """))
    # Inserta libros.
    conn.execute(text("""
        INSERT INTO Libros (titulo, autor_id, año_publicacion, genero) VALUES
        ('Temporada de caza para el león negro', 1, 2013, 'Novela contemporánea'),
        ('Autos usados', 2, 2012, 'Novela contemporánea'),
        ('Las batallas en el desierto', 3, 1981, 'Novela corta'),
        ('El complot mongol', 4, 1969, 'Novela negra')
    """))
    # Inserta préstamos.
    conn.execute(text("""
        INSERT INTO Prestamos (libro_id, fecha_prestamo, fecha_devolucion, usuario) VALUES
        (1, '2025-11-01', '2025-11-10', 'UsuarioA'),
        (2, '2025-11-05', NULL, 'UsuarioB'),
        (4, '2025-11-07', '2025-11-15', 'UsuarioC')
    """))


Como decía, las tablas contienen información de libros de autores que me gustan. Todos mexicanos.

## Consultas
Bien, ahora viene lo más interesante que es _SQL_ y sus consultas. Vamos a empezar con la consulta básica: `SELECT`. Con esta consulta indicamos qué vamos a extraer de una tabla. Empezaremos viendo la información completa de todas las tablas, la cual sustraemos con el símbolo `*`.

In [5]:
# Consulta autores.
autores = pd.read_sql_query("SELECT * FROM Autores", engine)
print("Ésta es la tabla 'autores'.")
print(autores)
print()
print('_______________________________________________________________________________________')
print()

# Consulta libros.
libros = pd.read_sql_query("SELECT * FROM Libros", engine)
print("Ésta es la tabla 'libros'.")
print(libros)
print()
print('_______________________________________________________________________________________')
print()

# Consulta préstamos.
prestamos = pd.read_sql_query("SELECT * FROM Prestamos", engine)
print("Ésta es la tabla 'préstamos'.")
print(prestamos)
print()

Ésta es la tabla 'autores'.
   id                    nombre    pais
0   1           Tryno Maldonado  México
1   2  Daniel Espartaco Sánchez  México
2   3       José Emilio Pacheco  México
3   4             Rafael Bernal  México

_______________________________________________________________________________________

Ésta es la tabla 'libros'.
   id                                titulo  autor_id  año_publicacion  \
0   1  Temporada de caza para el león negro         1             2013   
1   2                          Autos usados         2             2012   
2   3           Las batallas en el desierto         3             1981   
3   4                     El complot mongol         4             1969   

                 genero  
0  Novela contemporánea  
1  Novela contemporánea  
2          Novela corta  
3          Novela negra  

_______________________________________________________________________________________

Ésta es la tabla 'préstamos'.
   id  libro_id fecha_prestamo fec

Tres tablas con pocos registros porque esta nueva librería apenas empieza a almacenar la información en su base de datos. Vamos a empezar a resolver algunas preguntas. Las tablas contienen los libros y el nombre de los autores en dos tablas diferentes. Podemos unir tablas con la cláusula `JOIN`.

In [6]:
# Libros y nombre de su autor
df = pd.read_sql_query("""
    SELECT Libros.titulo, Autores.nombre 
    FROM Libros 
    JOIN Autores ON Libros.autor_id = Autores.id
""", engine)
print(df)

                                 titulo                    nombre
0  Temporada de caza para el león negro           Tryno Maldonado
1                          Autos usados  Daniel Espartaco Sánchez
2           Las batallas en el desierto       José Emilio Pacheco
3                     El complot mongol             Rafael Bernal


`JOIN` en por defecto un `INNER JOIN`, es decir, nos muestra combinaciones coincidentes según la llave `autor_id`. Traducido al español, esto quiere decir que unimos la tabla con base en su campo común (`autor_id`). Hemos descubierto los autores de los libros.

Ahora imaginemos que nos preguntan sobre un libro con un nombre un poco raro y difícil de memorizar que incluye la palabra 'negro'. ¿Qué podríamos hacer? Podemos buscar fragmentos en el título del libro con la cláusula `LIKE`:

In [7]:
# Libros que incluyen "negro" en el título
df = pd.read_sql_query("""
    SELECT titulo FROM Libros 
    WHERE titulo LIKE '%negro%'
""", engine)
print(df)

                                 titulo
0  Temporada de caza para el león negro


Después de la cláusula `LIKE` debemos poner el fragmento entre comillas y el símbolo `%`. Si sólo lo ponemos entre comillas, no estaríamos buscando un fragmento, sino una coincidencia literal, es decir, un libro con el título 'negro'. Como no tenemos ningún libro que se llame así, nuestra consulta no arrojaría resultados.

In [8]:
# Libros que incluyen "negro" en el título
df = pd.read_sql_query("""
    SELECT titulo FROM Libros 
    WHERE titulo LIKE 'negro'
""", engine)
print(df)

Empty DataFrame
Columns: [titulo]
Index: []


Muy bien. Ahora que hemos informado que el título del libro es _Temporada de caza para el leon negro_, nos han preguntado la cantidad total de préstamos hechos. Vamos a la tabla préstamos:

In [9]:
df = pd.read_sql_query("SELECT COUNT(*) AS cantidad_prestamos FROM Prestamos", engine)
print(df)

   cantidad_prestamos
0                   3


En esta consulta hemos utilizado la clásula `SELECT` para seleccionar y la clásula `COUNT` para contar el número de libros en la tabla `Prestamos`.

Después de esta consulta, debemos averiguar cuáles son los autores con libros prestados. La dificultad aumenta. Debemos unir la información de todas las tablas.

Primero seleccionaremos los nombres de los autores con `SELECT` y seguido de la cláusula `DISTINCT` (para no tener autores duplicados). La base de nuestra consulta será la tabla `Prestamos`.

Una vez hecha la selección, uniremos la tabla `Libros` con la tabla `Prestamos` con base en el campo `libro_id`. 

Finalmente unimos la tabla autores con libros con base en el mismo campo. Así obtenemos los nombres de los autores de cada libro prestado

In [10]:
df = pd.read_sql_query("""
    SELECT DISTINCT Autores.nombre 
    FROM Prestamos
    JOIN Libros ON Prestamos.libro_id = Libros.id
    JOIN Autores ON Libros.autor_id = Autores.id
""", engine)
print(df)

                     nombre
0           Tryno Maldonado
1  Daniel Espartaco Sánchez
2             Rafael Bernal


Ya sabemos qué autores son los que tienen libros con préstamo; se trata de tres y nuestros datos contienen cuatro autores. ¿Cuál es el autor que tiene no tiene un préstamo y cuál es el nombre del libro? Para descubrirlo debemos hacer algo parecido a la consulta anterior.

Primero seleccionamos el `Titulo` y el `Nombre`. Como base tendremos la tabla `Libros`.

Después uniremos autores con libros.

Finalmente usaremos la clásula `WHERE` que nos permite añadir condiciones. La condición `WHERE Libros.id NOT IN (SELECT libro_id FROM Prestamos)` significa 'selecciona los libros que _no estén registrados_ en la tabla préstamos'.

Como puedes ver, una parte de la consulta aparece entre paréntesis. Se trata de una subconsulta. Esta subconsulta obtiene todos los `libro_id` de la tabla préstamos (es decir, los libros prestados).

In [11]:
df = pd.read_sql_query("""
    SELECT Libros.titulo, Autores.nombre 
    FROM Libros
    JOIN Autores ON Libros.autor_id = Autores.id
    WHERE Libros.id NOT IN (SELECT libro_id FROM Prestamos)
""", engine)
print(df)


                        titulo               nombre
0  Las batallas en el desierto  José Emilio Pacheco


La consulta anterior fue un éxito. Introducimos la cláusula `WHERE` para añadir condiciones y las subconsultas, que nos permiten realizar búsqueda más complejas a través de filtrar registros usando datos de otra búsqueda.

## Conclusión
Demasiadas consultas por hoy. Hemos aprendido a hacer selecciones precisas a través de filtrar fragmentos, añadir condiciones, hacer subconsultas, unir tablas, etc. _SQL_ es una heramienta que exige —así como su nombre dice— un pensamiento *estructurado*. Si nos concentramos en entender la estructura de este lenguaje, estaremos del otro lado. En los siguientes cuadernos se profundizará más en esta herramienta.